# Load GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dependencies

In [ ]:
! pip install sentence-transformers

     |████████████████████████████████| 78 kB 3.5 MB/s 
     |████████████████████████████████| 3.4 MB 11.2 MB/s 
     |████████████████████████████████| 3.3 MB 36.3 MB/s 
     |████████████████████████████████| 1.2 MB 48.6 MB/s 
     |████████████████████████████████| 61 kB 577 kB/s 
     |████████████████████████████████| 895 kB 38.3 MB/s 
     |████████████████████████████████| 596 kB 36.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=4e12a5f2c6bb977c606d00a1cfc9c47fb4085710bf81c129cd7bda5a9eeb44d6
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Common

In [ ]:
import math
import os
import gzip
import csv
import torch
import json
from datetime import datetime
from tqdm import tqdm
from typing import Dict, List, Tuple, Callable
from torch.utils.data import DataLoader
from sentence_transformers import models, losses, util
from sentence_transformers import LoggingHandler, SentenceTransformer, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, LabelAccuracyEvaluator

In [ ]:
def loadDataset(filename: str, getScore: Callable[[Dict], float]) -> Tuple[List[InputExample], List[InputExample], List[InputExample]]:
    tr = []
    dev = []
    test = []
    with gzip.open(filename, 'rt', encoding='utf8') as f:
        for row in tqdm(csv.DictReader(f, delimiter='\t', quoting=csv.QUOTE_NONE)):
            sample = InputExample(texts=[row['sentence1'], row['sentence2']], label=getScore(row))
            sample.texts
            if row['split'] == 'train':
                tr.append(sample)
            elif row['split'] == 'dev':
                dev.append(sample)
            elif row['split'] == 'test':
                test.append(sample)
    return (tr, dev, test)

In [ ]:
nliPath = 'data/AllNLI.tsv.gz'
slsPath = 'data/stsbenchmark.tsv.gz'
if not os.path.exists(nliPath): util.http_get('https://sbert.net/datasets/AllNLI.tsv.gz', nliPath)
if not os.path.exists(slsPath): util.http_get('https://sbert.net/datasets/stsbenchmark.tsv.gz', slsPath)

  0%|          | 0.00/40.8M [00:00<?, ?B/s]

  0%|          | 0.00/392k [00:00<?, ?B/s]

In [ ]:
label2int = {"contradiction": 0, "neutral": 1, "entailment": 2 }
NLItr, NLIdev, NLItest = loadDataset(nliPath, lambda row: label2int[row['label']])
STStr, STSdev, STStest = loadDataset(slsPath, lambda row: (float(row['score']) / 2.5) - 1.0)

981382it [00:13, 70351.57it/s]
8628it [00:00, 98914.48it/s]


In [ ]:
modelName = 'distilbert-base-uncased'
batchSize = 16

# Regression

In [32]:
savePath = f'models/sbert_{modelName}_regression_STS'

In [33]:
embeddingModel = models.Transformer(modelName)
poolingModel = models.Pooling(
    embeddingModel.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False
)
model = SentenceTransformer(modules=[embeddingModel, poolingModel])

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
loss = losses.CosineSimilarityLoss(model=model)
dataloaderTr = DataLoader(STStr, shuffle=True, batch_size=batchSize)
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(STSdev, name='sts-dev')

In [ ]:
nEpochs = 10
model.fit(
    train_objectives=[(dataloaderTr, loss)],
    evaluator=evaluator,
    epochs=nEpochs,
    evaluation_steps=100,
    warmup_steps=math.ceil(len(dataloaderTr) * nEpochs  / 10),
    output_path=savePath
)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

In [ ]:
model = SentenceTransformer(savePath)
testEvaluator = EmbeddingSimilarityEvaluator.from_input_examples(STStest, name='sts-test')
testEvaluator(model, output_path=savePath)

# Classification

Training on NLI

In [ ]:
savePath = f'models/sbert_{modelName}_classification_NLI'

In [ ]:
embeddingModel = models.Transformer(modelName)
poolingModel = models.Pooling(
    embeddingModel.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False
)
model = SentenceTransformer(modules=[embeddingModel, poolingModel])

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=len(label2int))
dataloaderTr = DataLoader(NLItr, shuffle=True, batch_size=batchSize)
dataloaderDev = DataLoader(NLIdev, shuffle=False, batch_size=512) 
evaluator = LabelAccuracyEvaluator(dataloaderDev, name='nli-dev', softmax_model=loss)

In [11]:
nEpochs = 1
model.fit(
    train_objectives=[(dataloaderTr, loss)],
    evaluator=evaluator,
    epochs=nEpochs,
    evaluation_steps=2000,
    warmup_steps=math.ceil(len(dataloaderTr) * nEpochs  / 10),
    output_path=savePath
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/58880 [00:00<?, ?it/s]

In [13]:
model = SentenceTransformer(savePath).to('cuda:0')
dataloaderTest = DataLoader(NLItest, shuffle=False, batch_size=512) 
testEvaluator = LabelAccuracyEvaluator(dataloaderTest, name='nli-test', softmax_model=loss) 
testEvaluator(model, output_path=savePath)

0.7963471713471714

Fine-tuning on STS

In [16]:
savePath = f'models/sbert_{modelName}_classification_NLI-STS'

In [17]:
loss = losses.CosineSimilarityLoss(model=model).to('cuda:0')
dataloaderTr = DataLoader(STStr, shuffle=True, batch_size=batchSize)
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(STSdev, name='sts-dev')

In [18]:
nEpochs = 10
model.fit(
    train_objectives=[(dataloaderTr, loss)],
    evaluator=evaluator,
    epochs=nEpochs,
    evaluation_steps=100,
    warmup_steps=math.ceil(len(dataloaderTr) * nEpochs  / 10),
    output_path=savePath
)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

In [19]:
model = SentenceTransformer(savePath)
testEvaluator = EmbeddingSimilarityEvaluator.from_input_examples(STStest, name='sts-test')
testEvaluator(model, output_path=savePath)

0.7626878851160475

# Newspaper query

In [26]:
! mkdir -p embeddings

In [20]:
embedingsFilename = 'embeddings/embeddings.tensor'

In [21]:
if not os.path.exists('data/newspaper.json'):
  util.http_get('http://nicolatoscan.altervista.org/newspaper.json', 'data/newspaper.json')

  0%|          | 0.00/83.7M [00:00<?, ?B/s]

In [24]:
corpus = []
links = []
with open('data/newspaper.json', 'r') as f:
  corpus.extend([ json.loads(l)['headline'] for l in f ])
with open('data/newspaper.json', 'r') as f:
  links.extend([ json.loads(l)['link'] for l in f ])

In [27]:
def getCorpusEmbeding():
  if not os.path.exists(embedingsFilename):
    # Compute embeddings
    emb =  model.encode(corpus, convert_to_tensor=True)
    torch.save(emb, embedingsFilename)
    return emb
  else:
    # Load from file
    return torch.load(embedingsFilename)
embeddings = getCorpusEmbeding()

In [30]:
def query(query, k=5):
  cosScores = util.pytorch_cos_sim(
    model.encode(query, convert_to_tensor=True),
    embeddings
  )[0]
  res = torch.topk(cosScores, k=k)
  titles = [ ( score, corpus[idx], links[idx] ) for score, idx in zip(res[0], res[1]) ]
  for scores, title, link in titles:
    print(title, link)

In [31]:
query('President of the USA', k=10)

The Case For President Pence https://www.huffingtonpost.com/entry/the-case-for-president-pence_us_59224398e4b03b485cb262f1
The United Base Of America https://www.huffingtonpost.com/entry/the-united-base-of-america_us_59d4dabbe4b0da85e7f5ed2f
Come To Listen, Mr. President https://www.huffingtonpost.com/entry/come-to-listen-mr-president_us_59ca33c0e4b0f2df5e83b146
Joe Scarborough: Steve Bannon Is President Of The United States https://www.huffingtonpost.com/entry/steve-bannon-president-joe-scarborough_us_593175c0e4b0c242ca2344ab
From the Steps of the United States Supreme Court https://www.huffingtonpost.com/entry/from-the-steps-of-the-united-states-supreme-court_b_7182938.html
The 'President Of The United States' Made An Appearance At The Tonys https://www.huffingtonpost.com/entry/frank-underwood-president-tony-awards_us_593eab9be4b0b13f2c6c724a
Americans https://www.huffingtonpost.com/entry/post_us_5b9d5d4ae4b03a1dcc871566
About That American Idea https://www.huffingtonpost.comhttp://w

# Save model to GDrive

In [ ]:
! zip -r models.zip models

In [ ]:
! cp models.zip drive/MyDrive/SBERT/models.zip

In [ ]:
! zip -r embeddings.zip embeddings

In [ ]:
! cp embeddings.zip drive/MyDrive/SBERT/embeddings.zip

 # Download from GDrive

In [ ]:
! cp drive/MyDrive/SBERT/model.zip .

In [ ]:
! unzip model.zip

In [ ]:
! cp embeddings.zip drive/MyDrive/SBERT/embeddings.zip

In [ ]:
! unzip embeddings.zip